In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml

In [2]:
# source=requests.get(url=input('Enter the url of the tournament \n')).text

match_url_list = []

source_matchlist = requests.get(url='https://www.vlr.gg/event/matches/449/valorant-champions/?series_id=all').text

soup_matchlist = BeautifulSoup(source_matchlist,'lxml')

days = soup_matchlist.findAll('div', {'class':'wf-card'})

for d in range(1,len(days)):
    for i in days[d].findAll(href=True):
        match_url_list.append(i['href'])

In [3]:
match_url_list

['/51282/vision-strikers-vs-full-sense-valorant-champions-opening-d',
 '/51278/team-vikings-vs-crazy-raccoon-valorant-champions-opening-c',
 '/51283/fnatic-vs-cloud9-blue-valorant-champions-opening-d',
 '/51277/gambit-esports-vs-team-secret-valorant-champions-opening-c',
 '/51273/sentinels-vs-furia-valorant-champions-opening-b',
 '/51272/kr-esports-vs-team-liquid-valorant-champions-opening-b',
 '/51284/vision-strikers-vs-fnatic-valorant-champions-winners-d',
 '/51267/envy-vs-x10-crit-valorant-champions-opening-a',
 '/51279/gambit-esports-vs-team-vikings-valorant-champions-winners-c',
 '/51274/team-liquid-vs-sentinels-valorant-champions-winners-b',
 '/51266/acend-vs-keyd-stars-valorant-champions-opening-a',
 '/51280/team-secret-vs-crazy-raccoon-valorant-champions-elim-c',
 '/51275/kr-esports-vs-furia-valorant-champions-elim-b',
 '/51285/full-sense-vs-cloud9-blue-valorant-champions-elim-d',
 '/51268/acend-vs-envy-valorant-champions-winners-a',
 '/51269/keyd-stars-vs-x10-crit-valorant-cha

In [4]:
match_stats = []

for matchnum in range(len(match_url_list)):
    url = match_url_list[matchnum]
    
    source_match = requests.get(url='https://www.vlr.gg' + url).text
    
    soup_match = BeautifulSoup(source_match,'lxml')
    
    table_match = soup_match.findAll('table', {'class':'wf-table-inset mod-overview'})
    stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]
    series = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split("\n", 1)[1].strip()
    
    for a in [a for a in list(range(len(table_match))) if a not in [2,3]]:
        if a in [0,1]:
            map_num = 1
        else:
            map_num = a//2
        
        map_name = soup_match.findAll('div', {'class':'vm-stats-gamesnav-item js-map-switch'})[map_num-1].text.strip()[1:].strip()
        stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]
        team_name = soup_match.findAll('div', {'class':'wf-title-med'})[a % 2].text.strip()
        
        table = table_match[a]
        headers_match = []
        
        for i in table.find_all('th'):
            title_match = i.text.strip()
            headers_match.append(title_match)
            
        headers_match[0] = 'Player Name'
        headers_match[1] = 'Agent Name'
        headers_match[6] = 'K/D +/–'   
        headers_match[12] = 'FK/FD +/–'   
        df_match = pd.DataFrame(columns=headers_match)
        
        for row in table.find_all('tr')[1:]:
            data = row.find_all('td')
            row_data = [td.text.strip() for td in data]
            row_data[1] = str(data[1]).split('title="',1)[1].split('"',1)[0].title()
            length = len(df_match)
            df_match.loc[length] = row_data
        
        df_match['Team Name'] = team_name
        df_match['Map Name'] = map_name
        df_match['Map #'] = map_num
        df_match['Stage'] = stage
        df_match['Series'] = series
        
        match_stats.append(df_match)

In [5]:
result = pd.concat(match_stats).reset_index(drop=True)
result['Player Name'] = result['Player Name'].str.split("\n", 1).str[0].str.strip()
result['D'] = result['D'].str[1:-1]
result = result.apply(pd.to_numeric, errors='ignore')
result.drop('KAST', axis=1, inplace = True)

In [6]:
result

,Player Name,Agent Name,ACS,K,D,A,K/D +/–,ADR,HS%,FK,FD,FK/FD +/–,Team Name,Map Name,Map #,Stage,Series
0,stax,Breach,262,22,11,5,11,157,53%,2,2,0,Vision Strikers,Haven,1,Group Stage,Opening (D)
1,Rb,Skye,212,13,10,7,3,158,33%,4,2,2,Vision Strikers,Haven,1,Group Stage,Opening (D)
2,k1Ng,Killjoy,212,13,13,3,0,158,24%,3,2,1,Vision Strikers,Haven,1,Group Stage,Opening (D)
3,MaKo,Astra,209,13,8,6,5,154,24%,1,1,0,Vision Strikers,Haven,1,Group Stage,Opening (D)
4,BuZz,Jett,206,14,12,6,2,152,38%,0,1,-1,Vision Strikers,Haven,1,Group Stage,Opening (D)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705,d3ffo,Jett,271,19,18,1,1,176,29%,5,5,0,Gambit Esports,Split,5,Playoffs,Grand Final
706,Chronicle,Killjoy,212,14,15,3,-1,144,19%,2,2,0,Gambit Esports,Split,5,Playoffs,Grand Final
707,nAts,Viper,169,12,16,4,-4,111,18%,0,2,-2,Gambit Esports,Split,5,Playoffs,Grand Final
708,Redgar,Astra,125,9,17,5,-8,102,20%,0,3,-3,Gambit Esports,Split,5,Playoffs,Grand Final


In [7]:
result[result['Player Name'] == 'TenZ']

,Player Name,Agent Name,ACS,K,D,A,K/D +/–,ADR,HS%,FK,FD,FK/FD +/–,Team Name,Map Name,Map #,Stage,Series
101,TenZ,Jett,262,21,16,5,5,179,18%,4,3,1,Sentinels,Ascent,1,Group Stage,Opening (B)
110,TenZ,Jett,270,22,17,3,5,171,30%,5,2,3,Sentinels,Breeze,2,Group Stage,Opening (B)
120,TenZ,Jett,267,22,16,3,6,166,16%,5,0,5,Sentinels,Haven,3,Group Stage,Opening (B)
238,TenZ,Jett,163,14,21,4,-7,110,19%,6,4,2,Sentinels,Breeze,1,Group Stage,Winner's (B)
245,TenZ,Jett,369,21,6,2,15,211,19%,5,2,3,Sentinels,Bind,2,Group Stage,Winner's (B)
258,TenZ,Jett,141,12,20,2,-8,102,18%,4,5,-1,Sentinels,Split,3,Group Stage,Winner's (B)
402,TenZ,Jett,202,13,12,6,1,136,18%,5,5,0,Sentinels,Fracture,1,Group Stage,Decider (B)
411,TenZ,Jett,234,18,15,6,3,163,17%,5,1,4,Sentinels,Haven,2,Group Stage,Decider (B)
424,TenZ,Jett,119,9,20,7,-11,92,24%,1,7,-6,Sentinels,Split,3,Group Stage,Decider (B)


In [8]:
result[result['Agent Name'] == 'Jett'].groupby(['Player Name']).mean().sort_values('ACS', ascending=False)

,ACS,K,D,A,K/D +/–,ADR,FK,FD,FK/FD +/–,Map #
Player Name,,,,,,,,,,
Derke,305.500000,26.500000,14.750000,1.750000,11.750000,186.750000,6.500000,3.750000,2.750000,1.250000
leaf,299.800000,24.000000,18.000000,6.800000,6.000000,197.000000,7.000000,4.200000,2.800000,1.800000
heat,264.800000,17.600000,14.400000,1.600000,3.200000,174.800000,2.600000,3.200000,-0.600000,1.800000
Sushiboys,250.500000,20.000000,12.500000,3.000000,7.500000,168.500000,2.500000,1.000000,1.500000,1.000000
yay,242.714286,18.857143,15.285714,3.142857,3.571429,159.857143,4.285714,2.571429,1.714286,1.714286
DubsteP,237.666667,15.666667,13.666667,2.000000,2.000000,153.444444,4.000000,2.555556,1.444444,1.666667
cNed,231.454545,16.636364,14.727273,3.909091,1.909091,153.454545,3.636364,2.454545,1.181818,2.272727
xand,231.166667,17.666667,17.166667,3.833333,0.500000,145.666667,3.666667,3.500000,0.166667,2.000000
Patiphan,230.285714,16.428571,15.142857,4.142857,1.285714,147.857143,3.571429,2.714286,0.857143,2.000000
